In [4]:
import numpy as np


def main(x1, x2, x3):
    return x1 ** 2 + x2 ** 2 + x3 ** 2


def diff_f(x1, x2, x3):
    diff_f_ds = np.vstack((2 * x2, 2 * x3))
    diff_f_dd = np.array([2 * x1], dtype=float)
    return diff_f_ds, diff_f_dd


def cons(x1, x2, x3):
    h1 = x1 ** 2 / 4 + x2 ** 2 / 5 + x3 ** 2 / 25 - 1
    h2 = x1 + x2 - x3
    return h1, h2


def diff_h_dx(x1, x2, x3):

    dh_ds = np.array([[(2 / 5) * x2, (2 / 25) * x3], [1, -1]], dtype=float)
    dh_ds_inv = np.linalg.inv(dh_ds)
    dh_dd = np.vstack(((1 / 2) * x1, 1.))
    return dh_ds_inv, dh_dd, dh_ds


def diff_f_dd(x1, x2, x3):
    df_ds, df_dd = diff_f(x1, x2, x3)
    dh_ds_inv, dh_dd, _ = diff_h_dx(x1, x2, x3)
    return df_dd - np.matmul(df_ds.T, np.matmul(dh_ds_inv, dh_dd))


def solveh(x1, x2, x3):
    error = 1e-3
    # we are checking if the constraint is satisfied?
    h1, h2 = cons(x1, x2, x3)
    h = np.vstack((h1, h2))
    h_norm = np.linalg.norm(h)
    # assume the pseudo inverse
    while h_norm >= error:
        dh_inv, _, dh = diff_h_dx(x1, x2, x3)
        Lambda = 1
        ds = np.matmul(dh_inv, h)
        x2 = x2 - ds[0]
        x3 = x3 - ds[1]
        h1, h2 = cons(x1, x2, x3)
        h = np.vstack((h1, h2))
        h_norm = np.linalg.norm(h)
    return x1, x2, x3, h_norm


def l_s(x1, x2, x3):
    a = 1.  # initialize step size
    df = diff_f_dd(x1, x2, x3)

    def phi(a, x1, x2, x3, df): return main(x1, x2, x3) - a * 0.3 * \
        np.matmul(df, df.T)  # define phi as a search criterion

    def f_a(x1, x2, x3, a):
        df = diff_f_dd(x1, x2, x3)
        dh_ds_inv, dh_dd, _ = diff_h_dx(x1, x2, x3)

        x1 = x1 - a * df.flatten()
        ds = np.matmul(np.matmul(dh_ds_inv, dh_dd), df.T).flatten()
        x2 = x2 + a * ds[0]
        x3 = x3 + a * ds[1]
        return main(x1, x2, x3)

    while phi(a, x1, x2, x3, df) < f_a(x1, x2, x3, a):
        a = 0.5 * a
        df = diff_f_dd(x1, x2, x3)
    return a


eps = 1e-3  # criteria for termination
x1 = np.array([1.0], dtype=float)
x2 = np.array([2.0], dtype=float)
x3 = np.array([3.0], dtype=float)
iter = 0 

x1, x2, x3, _ = solveh(x1, x2, x3)

df_dd_norm = np.linalg.norm(diff_f_dd(x1, x2, x3))

while df_dd_norm >= eps:  
    a = l_s(x1, x2, x3)
    x1 = x1 - a * diff_f_dd(x1, x2, x3).flatten()
    dh_ds_inv, dh_dd, dh = diff_h_dx(x1, x2, x3)
    ds = a * np.matmul(np.matmul(dh_ds_inv, dh_dd),
                       diff_f_dd(x1, x2, x3).T).flatten()
    x2 = x2 + ds[0]
    x3 = x3 + ds[1]
    x1, x2, x3, _ = solveh(x1, x2, x3)
    df_dd_norm = np.linalg.norm(diff_f_dd(x1, x2, x3))

print('Solution is: X=({},{},{})'.format(x1, x2, x3))


Solution is: X=([-1.57329303],[1.37727404],[-0.19659334])


C:\Users\ggaut\AppData\Local\Temp\ipykernel_5772\3861636929.py:22: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  dh_ds = np.array([[(2 / 5) * x2, (2 / 25) * x3], [1, -1]], dtype=float)
